<a href="https://colab.research.google.com/github/RishabhRaj-SE/Machine_Learning_Model/blob/main/Customer_churn__codsoft.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Importing the dependencies


In [ ]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split,cross_val_score
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.preprocessing import OneHotEncoder
import pickle
import joblib
import warnings
warnings.filterwarnings('ignore')
from sklearn.preprocessing import LabelEncoder
from imblearn.over_sampling import SMOTE
from sklearn.tree import DecisionTreeClassifier
from xgboost import XGBClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.naive_bayes import GaussianNB

Data Loading and understanding

In [ ]:
df=pd.read_csv("/content/Churn_Modelling (1).csv")

In [ ]:
df.shape

(10000, 14)

In [ ]:
df.head()

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 14 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   RowNumber        10000 non-null  int64  
 1   CustomerId       10000 non-null  int64  
 2   Surname          10000 non-null  object 
 3   CreditScore      10000 non-null  int64  
 4   Geography        10000 non-null  object 
 5   Gender           10000 non-null  object 
 6   Age              10000 non-null  int64  
 7   Tenure           10000 non-null  int64  
 8   Balance          10000 non-null  float64
 9   NumOfProducts    10000 non-null  int64  
 10  HasCrCard        10000 non-null  int64  
 11  IsActiveMember   10000 non-null  int64  
 12  EstimatedSalary  10000 non-null  float64
 13  Exited           10000 non-null  int64  
dtypes: float64(2), int64(9), object(3)
memory usage: 1.1+ MB


In [ ]:
df=df.drop(columns=["RowNumber","CustomerId","Surname"])

In [ ]:
df.head()

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


In [ ]:
df["Gender"].unique()

array(['Female', 'Male'], dtype=object)

In [ ]:
print(df["Gender"].value_counts())

Gender
Male      5457
Female    4543
Name: count, dtype: int64


In [ ]:
df.head()

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


In [ ]:
df.head(10)

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0
5,645,Spain,Male,44,8,113755.78,2,1,0,149756.71,1
6,822,France,Male,50,7,0.00,2,1,1,10062.80,0
7,376,Germany,Female,29,4,115046.74,4,1,0,119346.88,1
8,501,France,Male,44,4,142051.07,2,0,1,74940.50,0
9,684,France,Male,27,2,134603.88,1,1,1,71725.73,0


In [ ]:
object_columns=df.select_dtypes(include="object").columns
print(object_columns)

Index(['Geography', 'Gender'], dtype='object')


In [ ]:
encoder={}

for col in object_columns:
   hot_encoder=LabelEncoder()
   df[col]=hot_encoder.fit_transform(df[col])
   encoder[col]=hot_encoder




with open("encoder.pkl","wb") as f:
    pickle.dump(encoder,f)


In [ ]:
df.head(20)

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,619,0,0,42,2,0.00,1,1,1,101348.88,1
1,608,2,0,41,1,83807.86,1,0,1,112542.58,0
2,502,0,0,42,8,159660.80,3,1,0,113931.57,1
3,699,0,0,39,1,0.00,2,0,0,93826.63,0
4,850,2,0,43,2,125510.82,1,1,1,79084.10,0
5,645,2,1,44,8,113755.78,2,1,0,149756.71,1
6,822,0,1,50,7,0.00,2,1,1,10062.80,0
7,376,1,0,29,4,115046.74,4,1,0,119346.88,1
8,501,0,1,44,4,142051.07,2,0,1,74940.50,0
9,684,0,1,27,2,134603.88,1,1,1,71725.73,0


In [ ]:
df["Geography"].value_counts()

,count
Geography,
0,5014
1,2509
2,2477


In [ ]:
df["Exited"].value_counts()

,count
Exited,
0,7963
1,2037


In [ ]:
x=df.drop(columns=["Exited"])
y=df["Exited"]

In [ ]:
x_train,x_test,y_train,y_test=train_test_split(x,y,test_size=0.2,random_state=2)

In [ ]:
print(y_train.value_counts())

Exited
0    6346
1    1654
Name: count, dtype: int64


In [ ]:
smote=SMOTE(random_state=2)

In [ ]:
x_train_smote,y_train_smote=smote.fit_resample(x_train,y_train)

In [ ]:
y_train_smote.shape

(12692,)

In [ ]:
y_train_smote.value_counts()

,count
Exited,
0,6346
1,6346


In [ ]:
# dictionary of models
models = {
    "Logistic Regression": LogisticRegression(random_state=42),
    "Support Vector Machine": SVC(random_state=42),
    "Naive Bayes": GaussianNB(),
    "Decision Tree": DecisionTreeClassifier(random_state=42),
    "Random Forest": RandomForestClassifier(random_state=42),
    "XGBoost": XGBClassifier(random_state=42)
}

In [ ]:
# dictionary to store the cross validation results
cv_scores = {}

# perform 5-fold cross validation for each model
for model_name, model in models.items():
  print(f"Training {model_name} with default parameters")
  scores = cross_val_score(model, x_train_smote, y_train_smote, cv=5, scoring="accuracy")
  cv_scores[model_name] = scores
  print(f"{model_name} cross-validation accuracy: {np.mean(scores):.2f}")
  print("-"*70)

Training Logistic Regression with default parameters
Logistic Regression cross-validation accuracy: 0.70
----------------------------------------------------------------------
Training Support Vector Machine with default parameters
Support Vector Machine cross-validation accuracy: 0.57
----------------------------------------------------------------------
Training Naive Bayes with default parameters
Naive Bayes cross-validation accuracy: 0.72
----------------------------------------------------------------------
Training Decision Tree with default parameters
Decision Tree cross-validation accuracy: 0.80
----------------------------------------------------------------------
Training Random Forest with default parameters
Random Forest cross-validation accuracy: 0.86
----------------------------------------------------------------------
Training XGBoost with default parameters
XGBoost cross-validation accuracy: 0.86
----------------------------------------------------------------------


In [ ]:
cv_scores

{'Logistic Regression': array([0.64119732, 0.76880662, 0.68282112, 0.69385343, 0.71158392]),
 'Support Vector Machine': array([0.57109098, 0.56833399, 0.57013396, 0.57762017, 0.56264775]),
 'Naive Bayes': array([0.68412761, 0.73020874, 0.72813239, 0.72892041, 0.73049645]),
 'Decision Tree': array([0.72784561, 0.79440725, 0.81993696, 0.80969267, 0.82427108]),
 'Random Forest': array([0.76171721, 0.86096889, 0.88810087, 0.8963751 , 0.88258471]),
 'XGBoost': array([0.77668373, 0.85506105, 0.88810087, 0.89243499, 0.89125296])}

In [ ]:
rfc=RandomForestClassifier(random_state=42)
rfc.fit(x_train_smote,y_train_smote)

RandomForestClassifier(random_state=42)

In [ ]:
y_test_pred=rfc.predict(x_test)

print("Accuracy Score:\n", accuracy_score(y_test, y_test_pred))
print("Confsuion Matrix:\n", confusion_matrix(y_test, y_test_pred))
print("Classification Report:\n", classification_report(y_test, y_test_pred))

Accuracy Score:
 0.813
Confsuion Matrix:
 [[1406  211]
 [ 163  220]]
Classification Report:
               precision    recall  f1-score   support

           0       0.90      0.87      0.88      1617
           1       0.51      0.57      0.54       383

    accuracy                           0.81      2000
   macro avg       0.70      0.72      0.71      2000
weighted avg       0.82      0.81      0.82      2000



In [ ]:
# save the trained model as a pickle file
model_data = {"model": rfc, "features_names": x.columns.tolist()}


with open("customer_churn_model.pkl", "wb") as f:
  pickle.dump(model_data, f)


Loading the model and building a predictive system


In [ ]:
# load teh saved model and the feature names

with open("customer_churn_model.pkl", "rb") as f:
  model_data = pickle.load(f)

loaded_model = model_data["model"]
feature_names = model_data["features_names"]

In [ ]:
prt(loaded_model)

RandomForestClassifier(random_state=42)


In [ ]:
print(feature_names)

['CreditScore', 'Geography', 'Gender', 'Age', 'Tenure', 'Balance', 'NumOfProducts', 'HasCrCard', 'IsActiveMember', 'EstimatedSalary']


In [ ]:
input_data={
           'CreditScore':376 ,
            'Geography':'Germany',
            'Gender':'Female',
            'Age':29,
            'Tenure':4,
            'Balance':11540.7,
            'NumOfProducts':4,
            'HasCrCard':1,
            'IsActiveMember':0,
            'EstimatedSalary':119346

}
input_df=pd.DataFrame(input_data,index=[0])



with open("encoder.pkl", "rb") as f:
  encoder = pickle.load(f)

for column,encoder in encoder.items():
  input_df[column]=encoder.transform(input_df[column])


columns_to_drop = ["RowNumber", "CustomerId", "Surname"]

for col in columns_to_drop:
  if col in input_df.columns:
    input_df.drop(columns=[col], inplace=True)

prediction=loaded_model.predict(input_df)

if prediction[0]==0:
  print("Customer will not Exit")
else:
  print("Customer will Exit")





Customer will Exit
